In [1]:
# credential code


In [2]:
import os
import requests
from requests.auth import HTTPBasicAuth

def download(purl: str, dest_folder: str):
    global username, password, site
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist
        
    url =f'{site}{purl}'
    

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)
    
    
    print(url)

    r = requests.get(url, stream=True, auth = HTTPBasicAuth(username, password))
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))



In [3]:
import yaml

def get_config(mconfig):
    global username, password, url_path, site, out_loc

    with open(mconfig, 'r') as rfile:
        cfg = yaml.safe_load(rfile)
        #print(cfg)
    try:
        # server_name = 'USGS-DDS'
        username = cfg['username']
        password = cfg['password']
        site = cfg['site']
        url_path = cfg['url']
        out_loc = cfg['output_location']
        output_name = cfg['outname']

        
    except ConnectionError:
        print('connection error occured')
        raise

if __name__ == "__main__":

    config = '/home/ec2-user/.config/wrangle.yaml'
    local_file = get_config(mconfig=config)


In [4]:
import zipfile
import os
def unzip_unlink(path_to_zip_file, directory_to_extract_to):
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)
    os.unlink(path_to_zip_file)

In [5]:
import requests
from bs4 import BeautifulSoup

# games_url = 'http://www.primarygames.com/mobile/category/all/'
games_url = "https://dds.cr.usgs.gov/highvolume/eviirs/Global_LST/expedited/VIIRS/2022/comp_293/"


def get_all_games():
    global username, password, url_path
    
    #print(username,password)

    soup = BeautifulSoup(requests.get(url_path, auth = HTTPBasicAuth(username, password)).text)
    #print(soup)
    schooltable = soup.find('table')
    my_links = schooltable.find_all("a", {"class": "linkUrl"})
    #print(my_links)
    for link in my_links:
        #print(link['href'])
        yield(link['href'])
 

if __name__ == '__main__':
    global out_loc, site
    for game in get_all_games():
        if game.endswith('.zip'):
            print(game)
            # download(game, out_loc)
            print('COMPLETED', game)
            just_file = game.split('/')[-1]
            full_path = f'{out_loc}{just_file}'
            print(full_path)
            unzip_unlink(full_path, out_loc)

/highvolume/eviirs/Global_LST/expedited/VIIRS/2022/comp_293/LS_eVSE_TEMP.2022.284-293.1KM.COMPRES.006.2022294053847.zip
COMPLETED /highvolume/eviirs/Global_LST/expedited/VIIRS/2022/comp_293/LS_eVSE_TEMP.2022.284-293.1KM.COMPRES.006.2022294053847.zip
/wsefs/tony/sdata/LS_eVSE_TEMP.2022.284-293.1KM.COMPRES.006.2022294053847.zip


In [6]:
#! mkdir -p /wsefs/tony/sdata
! ls -lh /wsefs/tony/sdata

total 4.1G
-rw-rw-r-- 1 ec2-user ec2-user  95K Dec  3 12:07 LS_eVSE_TEMP.2022.284-293.1KM.LST_ACQT.006.2022294042535.txt
-rw-rw-r-- 1 ec2-user ec2-user 517M Dec  3 12:07 LS_eVSE_TEMP.2022.284-293.1KM.LST_ANGL.006.2022294042535.tif
-rw-rw-r-- 1 ec2-user ec2-user 517M Dec  3 12:07 LS_eVSE_TEMP.2022.284-293.1KM.LST_EM14.006.2022294042535.tif
-rw-rw-r-- 1 ec2-user ec2-user 517M Dec  3 12:07 LS_eVSE_TEMP.2022.284-293.1KM.LST_EM15.006.2022294042535.tif
-rw-rw-r-- 1 ec2-user ec2-user 1.1G Dec  3 12:08 LS_eVSE_TEMP.2022.284-293.1KM.LST_NMBR.006.2022294042535.tif
-rw-rw-r-- 1 ec2-user ec2-user 517M Dec  3 12:08 LS_eVSE_TEMP.2022.284-293.1KM.LST_QLTY.006.2022294042535.tif
-rw-rw-r-- 1 ec2-user ec2-user 1.1G Dec  3 12:08 LS_eVSE_TEMP.2022.284-293.1KM.LST_TEMP.006.2022294042535.tif


In [7]:
! unzip -l /wsefs/tony/sdata/LS_eVSE_TEMP.2022.284-293.1KM.COMPRES.006.2022294053847.zip

unzip:  cannot find or open /wsefs/tony/sdata/LS_eVSE_TEMP.2022.284-293.1KM.COMPRES.006.2022294053847.zip, /wsefs/tony/sdata/LS_eVSE_TEMP.2022.284-293.1KM.COMPRES.006.2022294053847.zip.zip or /wsefs/tony/sdata/LS_eVSE_TEMP.2022.284-293.1KM.COMPRES.006.2022294053847.zip.ZIP.
